In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.offline as pyo
import numpy as np
import os
from tqdm.notebook import tqdm

In [2]:
file_lenght=-len(os.path.abspath(".").split("\\")[-1])
csv_path =  os.path.abspath(".")[:file_lenght]+"Data\\"

In [4]:
myData = pd.read_csv(csv_path+"Updated_Data_2.csv",header=0,engine='c',index_col=[0],low_memory=False)
myData.head()

E:\user\anaconda\envs\Forecasting_sales\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning:

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison



,ITINERARY_CODE,ITINERARY_NAME,PARTNER_CODE,PARTNERCATEGORY_CODE,PARTNERCATEGORYTYPE_CODE,VENDOR,SECTOR CODE,SECTOR NAME,REGION CODE,REGION NAME,...,UNITPRICE,SOLD QUANTITY,TURNOVER,DISCOUNT,GREGORIAN_DATE,HOLIDAY,SCOLAR_HOLIDAY,ISLAM_EVENT,TEMPERATURE,HUMIDITY
0,S02001,"Samedi, Centre ville 1",02/0002,M,M,R0201,201,EURL ADEMLAIT-CHLEF,2,CHLEF,...,88.000000,12.0,1056.000000,0.0,2017-11-25,0,0,None,55.59,0.54
1,S02001,"Samedi, Centre ville 1",02/0004,GM,GM,R0201,201,EURL ADEMLAIT-CHLEF,2,CHLEF,...,83.333333,36.0,2999.999988,0.0,2017-11-25,0,0,None,55.59,0.54
2,S02001,"Samedi, Centre ville 1",02/0004,GM,GM,R0201,201,EURL ADEMLAIT-CHLEF,2,CHLEF,...,88.000000,36.0,3168.000000,0.0,2017-11-25,0,0,None,55.59,0.54
3,S02001,"Samedi, Centre ville 1",02/0005,M,M,R0201,201,EURL ADEMLAIT-CHLEF,2,CHLEF,...,45.000000,12.0,540.000000,0.0,2017-11-25,0,0,None,55.59,0.54
4,S02001,"Samedi, Centre ville 1",02/0006,M,M,R0201,201,EURL ADEMLAIT-CHLEF,2,CHLEF,...,45.000000,24.0,1080.000000,0.0,2017-11-25,0,0,None,55.59,0.54


<h4>Detail about our Data and optimize the reading process for quick interactions</h4>

In [14]:
myData.info() #first look

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15692870 entries, 0 to 15692869
Data columns (total 23 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   ITINERARY_CODE            object 
 1   ITINERARY_NAME            object 
 2   PARTNER_CODE              object 
 3   PARTNERCATEGORY_CODE      object 
 4   PARTNERCATEGORYTYPE_CODE  object 
 5   VENDOR                    object 
 6   SECTOR CODE               int64  
 7   SECTOR NAME               object 
 8   REGION CODE               int64  
 9   REGION NAME               object 
 10  PRODUCT_CODE              object 
 11  PRODUCT_NAME              object 
 12  PRODUCTPAGE_CODE          object 
 13  UNITPRICE                 float64
 14  SOLD QUANTITY             float64
 15  TURNOVER                  float64
 16  DISCOUNT                  float64
 17  GREGORIAN_DATE            object 
 18  HOLIDAY                   int64  
 19  SCOLAR_HOLIDAY            int64  
 20  ISLAM_EVENT           

In [15]:
myData.info(memory_usage='deep') #deep look

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15692870 entries, 0 to 15692869
Data columns (total 23 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   ITINERARY_CODE            object 
 1   ITINERARY_NAME            object 
 2   PARTNER_CODE              object 
 3   PARTNERCATEGORY_CODE      object 
 4   PARTNERCATEGORYTYPE_CODE  object 
 5   VENDOR                    object 
 6   SECTOR CODE               int64  
 7   SECTOR NAME               object 
 8   REGION CODE               int64  
 9   REGION NAME               object 
 10  PRODUCT_CODE              object 
 11  PRODUCT_NAME              object 
 12  PRODUCTPAGE_CODE          object 
 13  UNITPRICE                 float64
 14  SOLD QUANTITY             float64
 15  TURNOVER                  float64
 16  DISCOUNT                  float64
 17  GREGORIAN_DATE            object 
 18  HOLIDAY                   int64  
 19  SCOLAR_HOLIDAY            int64  
 20  ISLAM_EVENT           

In [ ]:
for c in myData:
    k=myData[str(c)].unique()
    if len(k)<60 and str(c): #our data is composed of 15 million rows (category of 60 element is still saving memory usage)
        myData[str(c)]=myData[str(c)].astype("category")

In [17]:
myData.info(memory_usage='deep') #result

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15692870 entries, 0 to 15692869
Data columns (total 23 columns):
 #   Column                    Dtype   
---  ------                    -----   
 0   ITINERARY_CODE            object  
 1   ITINERARY_NAME            object  
 2   PARTNER_CODE              object  
 3   PARTNERCATEGORY_CODE      category
 4   PARTNERCATEGORYTYPE_CODE  category
 5   VENDOR                    object  
 6   SECTOR CODE               int64   
 7   SECTOR NAME               object  
 8   REGION CODE               category
 9   REGION NAME               category
 10  PRODUCT_CODE              category
 11  PRODUCT_NAME              category
 12  PRODUCTPAGE_CODE          category
 13  UNITPRICE                 category
 14  SOLD QUANTITY             float64 
 15  TURNOVER                  float64 
 16  DISCOUNT                  float64 
 17  GREGORIAN_DATE            object  
 18  HOLIDAY                   category
 19  SCOLAR_HOLIDAY            category
 20  

<p>We saved <b>7.0 GB</b> of memory</p>

<h2>Time series plot</h2>
<ul>
    <li>Daily Revenue all clients & all sectors (chiffre affaire par jour)</li>
    <li>Daily Revenue all product per sector (MCA par jour de tout produit par secteur)</li>
    <li>Daily Revenue all clients per Product range (MCA par jour selon la gamme produit de tout clients)</li>
</ul>

### Parameters as Daily-Monthly Progress, Menu Gamme Produit, Secteur-Region

<h4>Daily Revenue all clients & all sectors (chiffre affaire par jour)</h4>

In [128]:
cols=["SECTOR NAME","REGION NAME","PRODUCT_NAME","PRODUCTPAGE_CODE","TURNOVER","DISCOUNT","GREGORIAN_DATE"]
types={"SECTOR NAME": "category","REGION NAME": "category","PRODUCT_NAME": "category","PRODUCTPAGE_CODE": "category"}
myData = pd.read_csv(csv_path+"Updated_Data_2.csv",header=0,engine='c',usecols=cols,low_memory=False)
myData.head()

,SECTOR NAME,REGION NAME,PRODUCT_NAME,PRODUCTPAGE_CODE,TURNOVER,DISCOUNT,GREGORIAN_DATE
0,EURL ADEMLAIT-CHLEF,CHLEF,ENTIER,Laits Blancs,1056.000000,0.0,2017-11-25
1,EURL ADEMLAIT-CHLEF,CHLEF,DE 1L,Laits Blancs,2999.999988,0.0,2017-11-25
2,EURL ADEMLAIT-CHLEF,CHLEF,VIVA,Laits Blancs,3168.000000,0.0,2017-11-25
3,EURL ADEMLAIT-CHLEF,CHLEF,DE 50CL,Laits Blancs,540.000000,0.0,2017-11-25
4,EURL ADEMLAIT-CHLEF,CHLEF,DE 50CL,Laits Blancs,1080.000000,0.0,2017-11-25


In [5]:
myData.info(memory_usage='deep') #deep look

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15692870 entries, 0 to 15692869
Data columns (total 7 columns):
 #   Column            Dtype  
---  ------            -----  
 0   SECTOR NAME       object 
 1   REGION NAME       object 
 2   PRODUCT_NAME      object 
 3   PRODUCTPAGE_CODE  object 
 4   TURNOVER          object 
 5   DISCOUNT          float64
 6   GREGORIAN_DATE    object 
dtypes: float64(1), object(6)
memory usage: 6.1 GB


In [6]:
import cufflinks as cf               # third-party library for Pandas 
import plotly.graph_objs as go
%matplotlib inline
cf.go_offline()
pyo.init_notebook_mode(connected=True)
import ipywidgets as widgets
import itertools

#### Extract Attributes

In [129]:
sectors = myData[["SECTOR NAME","REGION NAME"]].drop_duplicates()
sectorDict = { region.upper() : [sector for sector in sectors[sectors["REGION NAME"]==region]["SECTOR NAME"]] for region in sectors["REGION NAME"].drop_duplicates().tolist()}
sectorDict

{'CHLEF': ['EURL ADEMLAIT-CHLEF'],
 'LAGHOUAT': ['DOUIHASNI AYACHE'],
 'OUM EL BOUAGHI': ['BOUCHAKOUR RAFIK'],
 'BATNA': ['BAAZIZI MOHAMED FOUZI', 'BAAZIZI MOHAMED FOUZI BARIKA'],
 'BEJAIA': ['MD CHERIF HAKIM', 'FOURAR AHCENE', 'IBRIR MERZOUK'],
 'BISKRA': ['EURL GOUBAA DISTRIBUTION'],
 'BECHAR': ['LAAMA MOUSSA'],
 'BLIDA': ['HADDAD ZAHIR'],
 'ALGER': ['MD CHERIF SAMIR',
  'IBRIR SLIMANE',
  'IBRIR ABDELHAK',
  'EURL MILK SODA FOOD',
  'IBRIR LARBI ALGER',
  'EURL CANDILAIT'],
 'BOUIRA': ['IBRIR ABDESLAM'],
 'TEBESSA': ['EURL NASR ECHARK'],
 'TLEMCEN': ['IBRIRKHELIL'],
 'TIARET': ['IBRIR SAMIR'],
 'TIZI OUZOU': ['SARL CANDY DISTRIBUTION', 'SARL CANDY DISTRIBUTION 2 -TO'],
 'DJELFA': ['BOUZIDI ABDELKRIM'],
 'JIJEL': ['SARL JGOAA'],
 'SETIF': ['EURL CANDY MILK', 'EURL DELICIOUS MILK'],
 'SAIDA': ['BELHOCINE Abdellatif'],
 'SKIKDA': ['HAMAZA LAKHDAR', 'FOURAR ALI'],
 'SIDI BEL ABBES': ['EURL EL TEWFIK'],
 'ANNABA': ['EURL ILYES FOOD', 'ANNABA 2'],
 'GUELMA': ['AZERINE YACINE', 'TABOU HAMI

In [130]:
productGamme = myData[["PRODUCT_NAME","PRODUCTPAGE_CODE"]].drop_duplicates()
productDict = { gamme.lower() : [product.lower() for product in productGamme[productGamme["PRODUCTPAGE_CODE"]==gamme]["PRODUCT_NAME"]] for gamme in productGamme["PRODUCTPAGE_CODE"].drop_duplicates().tolist()}
productDict

{'laits blancs': ['entier',
  'de 1l',
  'viva',
  'de 50cl',
  'silhouette',
  'de 1l sans lactose'],
 'laits aromatisés': ['choco 20cl',
  'choco 125 ml',
  'fraise 125 ml',
  'candy caramel 125 ml',
  'choco 1l',
  'fraise 20cl',
  'banane 125 ml'],
 'laits & jus': ['twist o-a 20cl',
  'twist o-f-b 20cl',
  'twist o-f-b 1l',
  'twist o-a 1l',
  'twist o-m 20cl',
  'twist p-a 20cl',
  'twist o-m 1l',
  'twist p-a 1l',
  'twist o-f-b 125ml',
  'twist o-a 125ml'],
 'boissons': ['cocktail 1l',
  'orange 1l',
  'citronnade',
  'nectar de grenade',
  'cocktail 20cl',
  'orange 20cl'],
 'le maitre cuisinier': ['preparation culinaire'],
 'le maitre glacier': ['ice cream - vanille',
  'ice cream - chocolat',
  'ice cream - fraise'],
 'poudre etui': ['etui 500g']}

In [79]:
%%timeit
myData["GREGORIAN_DATE"].str.slice(stop=7).drop_duplicates().sort_values()

4.32 s ± 99.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [80]:
%%timeit
myData["GREGORIAN_DATE"].map(lambda x: x[:7]).drop_duplicates().sort_values()

3.89 s ± 45.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [82]:
%%timeit
myData["GREGORIAN_DATE"].drop_duplicates().str.slice(stop=7).drop_duplicates().sort_values()

223 ms ± 4.91 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [81]:
%%timeit
myData["GREGORIAN_DATE"].drop_duplicates().map(lambda x: x[:7]).drop_duplicates().sort_values()

217 ms ± 4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [133]:
NewDataStock = myData
NewDataStock["GREGORIAN_MONTH"] = myData["GREGORIAN_DATE"].map(lambda x: x[:7])
NewDataStock.head(1)

,SECTOR NAME,REGION NAME,PRODUCT_NAME,PRODUCTPAGE_CODE,TURNOVER,DISCOUNT,GREGORIAN_DATE,GREGORIAN_MONTH
0,EURL ADEMLAIT-CHLEF,CHLEF,ENTIER,Laits Blancs,1056.0,0.0,2017-11-25,2017-11


In [134]:
layout = go.Layout(
                    title="Serie temporelles - Ventes de produits laitiers",
                    xaxis= {"title": "Time"},
                    yaxis= {"title": "per 1M dinar"}
                    )
def DataDisp(Mode,productGamme,regionSlider,promotion):
    
    data = []
     
    for gamme,region in tqdm(list(itertools.product(productGamme,regionSlider))):
        y = NewDataStock 
        if gamme != 'tous':
            y= y[y["PRODUCTPAGE_CODE"].str.lower().str.match(gamme)]
        if region != 'TOUS':
            y= y[y["REGION NAME"].str.upper().str.match(region)]

        if Mode=='Monthly':
            x= NewDataStock["GREGORIAN_MONTH"].drop_duplicates().sort_values()
            if promotion:
                y2= y.groupby("GREGORIAN_MONTH")["DISCOUNT"].sum()/1000000
            y= y.groupby("GREGORIAN_MONTH")["TURNOVER"].sum()/1000000
        else:
            x= NewDataStock["GREGORIAN_DATE"].drop_duplicates().sort_values()
            if promotion:
                y2= y.groupby("GREGORIAN_MONTH")["DISCOUNT"].sum()/1000000
            y= y.groupby("GREGORIAN_DATE")["TURNOVER"].sum()/1000000

        data.append(
            go.Scatter(
                x = x,
                y = y,
                mode="lines",
                name=f'CAF -- PRD:{gamme}, REG:{region}, {Mode}',
            )
        )
        
        if promotion:
            data.append(
                go.Scatter(
                    x = x,
                    y = y2,
                    mode="lines",
                    name=f'Promo -- PRD:{gamme}, REG:{region}, {Mode}',
                )
            )
    fig = go.Figure(data=data,layout=layout)
    pyo.iplot(fig)

In [135]:
Mode = widgets.RadioButtons(
    options=['Daily', 'Monthly'],
    value='Daily', # Defaults to daily
    layout={'width': 'max-content'}, 
    description='Mode:',
    disabled=False
)


promotion = widgets.Checkbox(
    value=False,
    description='Promotion',
    disabled=False
)


productGamme = widgets.SelectMultiple(
    options=['tous']+list(productDict.keys()),
    value=['tous'],
    rows=4,
    description='Gamme',
    disabled=False
)

regionSlider = widgets.SelectMultiple(
    options=['TOUS']+list(sectorDict.keys()),
    value=['TOUS'],
    rows=4,
    description='Region',
    disabled=False
)
        
widgets.interactive(DataDisp,Mode=Mode,productGamme=productGamme,regionSlider=regionSlider,promotion=promotion)

interactive(children=(RadioButtons(description='Mode:', layout=Layout(width='max-content'), options=('Daily', …

### Seasonability

In [ ]:
cols=["SECTOR NAME","REGION NAME","TURNOVER","DISCOUNT","GREGORIAN_DATE","VENDOR"]
types={"SECTOR NAME": "category","REGION NAME": "category"}
myData2 = pd.read_csv(csv_path+"Updated_Data_2.csv",header=0,engine='c',usecols=cols,low_memory=False)
myData2.head()

In [173]:
x = [f'-{i:02d}-' for i in range(1,13)]
y = [myData2[myData2["GREGORIAN_DATE"].str.contains(sub)]["TURNOVER"].sum()/3 for sub in tqdm(x)]
layout = go.Layout(title="Seasonability behaviours")
data=[]
data.append(go.Scatter(
            x = x,
            y = y,
            mode="lines",
            name="mean seasonability",))
for year in (2016,2017,2018):
    data.append(go.Scatter(
            x = x,
            y = [myData2[myData2["GREGORIAN_DATE"].str.match(str(year)+sub)]["TURNOVER"].sum() for sub in tqdm(x)],
            mode="lines",
            name=f"{year} seasonability",))

fig = go.Figure(data=data,layout=layout)
pyo.iplot(fig)

### Comportement vis à vis les vacances

In [54]:
cols=["REGION NAME","PRODUCT_NAME","PRODUCTPAGE_CODE","GREGORIAN_DATE","HOLIDAY","SCOLAR_HOLIDAY","ISLAM_EVENT","TURNOVER","VENDOR"]
types={column : "category" for column in cols[:7]}
myData3 = pd.read_csv(csv_path+"Updated_Data_2.csv",header=0,engine='c',usecols=cols,low_memory=False,encoding = 'utf-8',)
myData3.head()

,VENDOR,REGION NAME,PRODUCT_NAME,PRODUCTPAGE_CODE,TURNOVER,GREGORIAN_DATE,HOLIDAY,SCOLAR_HOLIDAY,ISLAM_EVENT
0,R0201,CHLEF,ENTIER,Laits Blancs,1056.000000,2017-11-25,0,0,None
1,R0201,CHLEF,DE 1L,Laits Blancs,2999.999988,2017-11-25,0,0,None
2,R0201,CHLEF,VIVA,Laits Blancs,3168.000000,2017-11-25,0,0,None
3,R0201,CHLEF,DE 50CL,Laits Blancs,540.000000,2017-11-25,0,0,None
4,R0201,CHLEF,DE 50CL,Laits Blancs,1080.000000,2017-11-25,0,0,None


In [4]:
myData3.info(memory_usage='deep') #deep look

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15692870 entries, 0 to 15692869
Data columns (total 9 columns):
 #   Column            Dtype  
---  ------            -----  
 0   VENDOR            object 
 1   REGION NAME       object 
 2   PRODUCT_NAME      object 
 3   PRODUCTPAGE_CODE  object 
 4   TURNOVER          float64
 5   GREGORIAN_DATE    object 
 6   HOLIDAY           int64  
 7   SCOLAR_HOLIDAY    int64  
 8   ISLAM_EVENT       object 
dtypes: float64(1), int64(2), object(6)
memory usage: 6.2 GB


In [22]:
y=myData3[["GREGORIAN_DATE","PRODUCTPAGE_CODE","ISLAM_EVENT","SCOLAR_HOLIDAY","HOLIDAY","TURNOVER"]]
y_islam=y[y["ISLAM_EVENT"]!="None"].groupby("GREGORIAN_DATE")["TURNOVER"].sum()/1000000
y_scolar=y[y["SCOLAR_HOLIDAY"]!=0].groupby("GREGORIAN_DATE")["TURNOVER"].sum()/1000000
y_holiday=y[y["HOLIDAY"]!=0].groupby("GREGORIAN_DATE")["TURNOVER"].sum()/1000000
y=y.groupby("GREGORIAN_DATE")["TURNOVER"].sum()/1000000
x=myData3["GREGORIAN_DATE"].drop_duplicates().sort_values().tolist()

In [43]:
x_scolar=y_scolar.index

In [48]:
y=myData3[["GREGORIAN_DATE","PRODUCTPAGE_CODE","ISLAM_EVENT","SCOLAR_HOLIDAY","HOLIDAY","TURNOVER"]]

y=y.groupby("GREGORIAN_DATE")["TURNOVER"].sum()/1000000
x=y.index
data=[go.Scatter(
            x =x,
            y = y,
            mode="lines",
            name=f"Time Serie",),
     go.Scatter(
            x =x_scolar,
            y = y_scolar,
            mode="markers",
            name=f"Time Serie",),]
layout = go.Layout(title="Holidays behaviour")
fig = go.Figure(data=data,layout=layout)
pyo.iplot(fig)

In [64]:
myData3.loc[myData3["PRODUCTPAGE_CODE"].str.lower().str.match('laits aromatisã\x83â©s'),"PRODUCTPAGE_CODE"]='laits aromatisés'

In [65]:
productGamme = myData3[["PRODUCT_NAME","PRODUCTPAGE_CODE"]].drop_duplicates()
productDict = { gamme.lower() : [product.lower() for product in productGamme[productGamme["PRODUCTPAGE_CODE"]==gamme]["PRODUCT_NAME"]] for gamme in productGamme["PRODUCTPAGE_CODE"].drop_duplicates().tolist()}
productDict

{'laits blancs': ['entier',
  'de 1l',
  'viva',
  'de 50cl',
  'silhouette',
  'de 1l sans lactose',
  'de 50cl'],
 'laits aromatisés': ['choco 20cl',
  'choco 125 ml',
  'fraise 125 ml',
  'candy caramel 125 ml',
  'choco 1l',
  'fraise 20cl',
  'banane 125 ml',
  'r16054'],
 'laits & jus': ['twist o-a 20cl',
  'twist o-f-b 20cl',
  'twist o-f-b 1l',
  'twist o-a 1l',
  'twist o-m 20cl',
  'twist p-a 20cl',
  'twist o-m 1l',
  'twist p-a 1l',
  'twist o-f-b 125ml',
  'twist o-a 125ml'],
 'boissons': ['cocktail 1l',
  'orange 1l',
  'citronnade',
  'nectar de grenade',
  'cocktail 20cl',
  'orange 20cl'],
 'le maitre cuisinier': ['preparation culinaire'],
 'le maitre glacier': ['ice cream - vanille',
  'ice cream - chocolat',
  'ice cream - fraise'],
 'poudre etui': ['etui 500g']}

In [67]:
productGamme = widgets.Select(
    options=['tous']+list(productDict.keys()),
    value='tous',
    rows=4,
    description='Gamme de produit',
    disabled=False
)

def Events(product):
    
    y=myData3[["GREGORIAN_DATE","PRODUCTPAGE_CODE","ISLAM_EVENT","SCOLAR_HOLIDAY","HOLIDAY","TURNOVER"]]
    if product != 'tous':
        y=y[y["PRODUCTPAGE_CODE"].str.lower().str.match(product)]
    
    y_islam=y[y["ISLAM_EVENT"]!="None"].groupby("GREGORIAN_DATE")["TURNOVER"].sum()/1000000
    x_islam=y_islam.index
    
    y_scolar=y[y["SCOLAR_HOLIDAY"]!=0].groupby("GREGORIAN_DATE")["TURNOVER"].sum()/1000000
    x_scolar=y_scolar.index

    y_holiday=y[y["HOLIDAY"]!=0].groupby("GREGORIAN_DATE")["TURNOVER"].sum()/1000000
    x_holiday=y_holiday.index
    
    y=y.groupby("GREGORIAN_DATE")["TURNOVER"].sum()/1000000
    x=y.index
    
    data=[go.Scatter(
            x = x_islam,
            y = y_islam,
            mode="markers",
            name=f"Islam events",),
          go.Scatter(
            x = x_scolar,
            y = y_scolar,
            mode="markers",
            name=f"Scolar holidays",),
          go.Scatter(
            x = x_holiday,
            y = y_holiday,
            mode="markers",
            name=f"Holidays",),
          go.Scatter(
            x = x,
            y = y,
            mode="lines",
            name=f"Time Serie",),]
    layout = go.Layout(title=f"Holidays behaviour - {product}")
    fig = go.Figure(data=data,layout=layout)
    pyo.iplot(fig)

widgets.interactive(Events,product = productGamme)

interactive(children=(Select(description='Gamme de produit', options=('tous', 'laits blancs', 'laits aromatisé…

### Clients par secteur ( pourcentage de clients )

In [148]:
print(f"Nombre total de clients est : {len(myData2['VENDOR'].drop_duplicates().tolist())}")
for annee in (2016,2017,2018):    
    data = myData2[myData2["GREGORIAN_DATE"].str.match(str(annee))]['VENDOR'].drop_duplicates().tolist()
    print(f"Nombre de clients en {annee} est : {len(data)}")

Nombre total de clients est : 480
Nombre de clients en 2016 est : 223
Nombre de clients en 2017 est : 264
Nombre de clients en 2018 est : 467


In [136]:
sectors = myData2[["SECTOR NAME","REGION NAME","VENDOR"]].drop_duplicates()
sectorDict = { region.upper() : {sector.upper(): {client.upper() for client in sectors[sectors["SECTOR NAME"]==sector]["VENDOR"]} for sector in sectors[sectors["REGION NAME"]==region]["SECTOR NAME"]} for region in sectors["REGION NAME"].drop_duplicates().tolist()}
sectorDict

{'CHLEF': {'EURL ADEMLAIT-CHLEF': {'R0201',
   'R0202',
   'R0203',
   'R0204',
   'R0205',
   'R0206'}},
 'LAGHOUAT': {'DOUIHASNI AYACHE': {'R0301', 'R0302', 'R0303', 'R0304'}},
 'OUM EL BOUAGHI': {'BOUCHAKOUR RAFIK': {'R0401',
   'R0402',
   'R04021',
   'R04022',
   'R04023',
   'R04024',
   'R04025',
   'R0403',
   'R0404',
   'R0405'}},
 'BATNA': {'BAAZIZI MOHAMED FOUZI': {'R0501',
   'R0502',
   'R0503',
   'R0504',
   'R0505',
   'R0506',
   'R0507',
   'R0508',
   'R0509',
   'R0510',
   'R0511',
   'R0512'},
  'BAAZIZI MOHAMED FOUZI BARIKA': {'R05021',
   'R05022',
   'R05023',
   'R0503',
   'R0504',
   'R0506',
   'R0512'}},
 'BEJAIA': {'MD CHERIF HAKIM': {'1',
   '15',
   '4',
   '5',
   '6',
   '7',
   'R06008',
   'R06031',
   'R06032',
   'R06033',
   'R06034',
   'R06035',
   'R06036',
   'R06037'},
  'FOURAR AHCENE': {'1',
   '15',
   '3',
   '5',
   '6',
   '7',
   'R06008',
   'R06041',
   'R06042',
   'R06043',
   'R06044',
   'R06045',
   'R06046',
   'R06047',
   

In [137]:
import plotly.express as px

def MoneyDisp(regions):
    List = ["ALGERIE"]
    REGIONS = [""]
    Value = [0]
    for region in tqdm(list(regions)[:10]):
        List.append(region)
        REGIONS.append("ALGERIE")
        Value.append(0)
        for secteur in sectorDict[region].keys():
            List.append(secteur)
            REGIONS.append(region)
            Value.append(myData2[myData2["SECTOR NAME"]==secteur]["TURNOVER"].sum())     
            
    data = dict(
        character=List,
        parent=REGIONS,
        value=Value
        )

    fig =px.sunburst(
        data,
        names='character',
        parents='parent',
        values='value',
    )
    fig = go.Figure(data=fig)
    pyo.iplot(fig)

In [138]:
region_name = widgets.SelectMultiple(
    options=list(sectorDict.keys()),
    value=["ALGER"],
    rows=4,
    description='Region',
    disabled=False
)
widgets.interactive(MoneyDisp,regions=region_name)

interactive(children=(SelectMultiple(description='Region', index=(8,), options=('CHLEF', 'LAGHOUAT', 'OUM EL B…

In [58]:
def test(region,population):
    print(f'les secteur sont {list(sectorDict[region])} qui est situe a la region {region}, on a choisis une population de {population} pourcent')
    
widgets.interactive(test,region=region_name,population=Population)

interactive(children=(Select(description='Region', index=8, options=('CHLEF', 'LAGHOUAT', 'OUM EL BOUAGHI', 'B…

### Granularité d'étude : Clients par REGION

In [144]:
N = len(sectorDict.keys())     # Number of boxes
c = ['hsl('+str(h)+',50%'+',50%)' for h in np.linspace(0, 360, N)] #colors

fig = go.Figure(data=[go.Box(
    y= myData2[myData2["REGION NAME"]==list(sectorDict.keys())[i]].groupby(["VENDOR"])["TURNOVER"].sum().tolist(),
    marker_color=c[i],
    name=list(sectorDict.keys())[i],
    ) for i in range(N)])

# format the layout
fig.update_layout(
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(zeroline=False, gridcolor='white'),
    paper_bgcolor='rgb(233,233,233)',
    plot_bgcolor='rgb(233,233,233)',
)

fig.show()

### Granularité : client par secteur

In [145]:
def ClientDisp(region,choix):
    df = myData2[myData2["REGION NAME"]=="ALGER"].groupby(["VENDOR","SECTOR NAME"]).sum().reset_index()
    fig = px.box(df, x="SECTOR NAME", y=choix, points="all",color = "SECTOR NAME")
    fig = go.Figure(data=fig)
    pyo.iplot(fig)

In [126]:
region_name = widgets.SelectMultiple(
    options=list(sectorDict.keys()),
    value=["ALGER"],
    rows=4,
    description='Region',
    disabled=False
)

choix = widgets.RadioButtons(
    options=['TURNOVER', 'DISCOUNT'],
    value='TURNOVER', # Defaults to turnover
    layout={'width': 'max-content'}, 
    description='Mode:',
    disabled=False
)

widgets.interactive(ClientDisp,region=region_name,choix=choix)

interactive(children=(SelectMultiple(description='Region', index=(8,), options=('CHLEF', 'LAGHOUAT', 'OUM EL B…